<a href="https://colab.research.google.com/github/xarvel/DataScience/blob/master/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import tensorflow as tf

import numpy as np
import os
import time

In [13]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [14]:
def text_stats(text):
  sample_size = 250
  print(f'Sample {sample_size} characters:')
  print('-' * 80)
  # Take a look at the first 250 characters in text
  print(text[:sample_size])
  print('-' * 80)
  # length of text is the number of characters in it
  print(f'Length of text: {len(text)} characters')
  # The unique characters in the file
  vocab = sorted(set(text))
  print(f'{len(vocab)} unique characters')

  return vocab

In [15]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
vocab = text_stats(text)

Sample 250 characters:
--------------------------------------------------------------------------------
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

--------------------------------------------------------------------------------
Length of text: 1115394 characters
65 unique characters


In [16]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab),
    mask_token=None
)
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(),
    invert=True,
    mask_token=None
)

In [17]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [18]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [19]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [20]:
seq_length = 100

In [21]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [22]:
sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

In [23]:
dataset = sequences.map(split_input_target)

In [24]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [25]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)


dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [26]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [27]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [28]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
)

In [29]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [30]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [32]:
sampled_indices

array([17, 48, 26, 50,  6, 17, 32, 44, 29, 65, 55, 63, 50, 61, 11, 17, 14,
       20, 54, 13, 32, 38, 12, 49, 59, 34, 32, 31, 10, 20, 47, 42,  0, 55,
        9, 12,  9, 24, 52,  1,  8, 36, 23, 18, 20, 61, 58, 25, 55, 58, 14,
       58, 63, 10, 61, 15, 11, 26, 60, 35, 37, 40, 64, 36, 45, 65, 11,  0,
       61, 33, 16, 27, 52, 26, 19, 45, 40, 28, 37, 58, 26, 37, 43, 45, 48,
       62,  1, 41, 31, 30, 36, 30, 15, 13, 51, 59, 21, 59, 43, 44])

In [33]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b's,\nHad time cohered with place or place with wishing,\nOr that the resolute acting of your blood\nCoul'

Next Char Predictions:
 b"DiMk'DSePzpxkv:DAGo?SY;jtUSR3Ghc[UNK]p.;.Km\n-WJEGvsLpsAsx3vB:MuVXayWfz:[UNK]vTCNmMFfaOXsMXdfiw\nbRQWQB?ltHtde"


In [34]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [35]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.189001, shape=(), dtype=float32)


In [36]:
tf.exp(example_batch_mean_loss).numpy()

65.95687

In [37]:
model.compile(optimizer='adam', loss=loss)

In [38]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [39]:
EPOCHS = 20

In [40]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 15s 55ms/step - loss: 2.7351
Epoch 2/20
172/172 [==============================] - 12s 54ms/step - loss: 1.9989
Epoch 3/20
172/172 [==============================] - 11s 56ms/step - loss: 1.7189
Epoch 4/20
172/172 [==============================] - 11s 57ms/step - loss: 1.5545
Epoch 5/20
172/172 [==============================] - 12s 57ms/step - loss: 1.4538
Epoch 6/20
172/172 [==============================] - 13s 58ms/step - loss: 1.3857
Epoch 7/20
172/172 [==============================] - 11s 56ms/step - loss: 1.3325
Epoch 8/20
172/172 [==============================] - 12s 57ms/step - loss: 1.2880
Epoch 9/20
172/172 [==============================] - 13s 57ms/step - loss: 1.2449
Epoch 10/20
172/172 [==============================] - 13s 58ms/step - loss: 1.2063
Epoch 11/20
172/172 [==============================] - 11s 57ms/step - loss: 1.1673
Epoch 12/20
172/172 [==============================] - 11s 57ms/step - loss: 1.1268
E

In [41]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [42]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [43]:
def predict_text(start_text, size):
  start = time.time()
  states = None
  next_char = tf.constant([start_text])
  result = [next_char]

  for n in range(size):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

  result = tf.strings.join(result)
  end = time.time()
  print(result[0].numpy().decode('utf-8'), '\n\n' + '_' * 80)
  print('\nRun time:', end - start)

In [44]:
predict_text('ROMEO:', 1000)

ROMEO:
The more most joys by the it; and struck me, the
point.

SICINIUS:
Miepty, sir, is it
Be new concluded: say'st thou turn away?

DUKE VINCENTIO:
Not I to live this shame but lately Polperee,
Even in every maje thy self-shining rabber,
Whose denitest of my blood from you indeed
That murderers immediately in notels gall;
Our certain with you: he hath forson, as it is
Acquaint them unto my claim:
Come, masters slain!

Second Senator:
Now, one kiss is but health!

FRIAR LAURENCE:

PETRUCHIO:
Why, most hidest thou not dear lord?

EDWARD:
I pray you all, I will go along,
But make halm'd more.

First Citizen:
No, both he in his short.

TRANIO:
Sir, mark you to disparse that, infirmity of speaking.
Seek for the eldest darks that stood and looker on.

WARWICK:
Except them, to the robes of my course, young
Apon to be disguised in every pat,
A market-place of marriage makes
Title all thy souls of any place. For John Northman,
To stain thy sort. I dare both myself,
Prick for one change of pr

In [45]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [46]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [47]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
)

In [48]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [49]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 14s 58ms/step - loss: 2.7133


In [50]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1923
Epoch 1 Batch 50 Loss 2.0602
Epoch 1 Batch 100 Loss 1.9197
Epoch 1 Batch 150 Loss 1.8556

Epoch 1 Loss: 1.9859
Time taken for 1 epoch 12.35 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.8335
Epoch 2 Batch 50 Loss 1.7336
Epoch 2 Batch 100 Loss 1.7230
Epoch 2 Batch 150 Loss 1.6472

Epoch 2 Loss: 1.7132
Time taken for 1 epoch 11.47 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.6209
Epoch 3 Batch 50 Loss 1.5901
Epoch 3 Batch 100 Loss 1.5165
Epoch 3 Batch 150 Loss 1.5098

Epoch 3 Loss: 1.5563
Time taken for 1 epoch 12.23 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4845
Epoch 4 Batch 50 Loss 1.4139
Epoch 4 Batch 100 Loss 1.4362
Epoch 4 Batch 150 Loss 1.4393

Epoch 4 Loss: 1.4581
Time taken for 1 epoch 11.17 sec
_____________________________________________________________________